In [193]:
from attr.validators import max_len

from RAG.llm.embedding import generate_embedding

from setuptools.command.alias import alias
from sympy.codegen.cxxnodes import using

# from retriever import retrieve_documents
# from llm.model import call_llm
from RAG.utils.pdf_loader import pdf_load
from RAG.utils.preprocess import clean_text, chunk_text
from RAG.database.milvus_connector import connect_to_milvus
from RAG.database.vector_store import save_to_milvus
from RAG.database.milvus_connector import *

# Connect Milvus

In [194]:
connect_to_milvus()

Connected to Milvus-tailorlink


# Create Collection

In [195]:
# embedding = generate_embedding(['Hi'])

In [196]:
# dense_dim = 1024

In [197]:
# from pymilvus import (
#     connections,
#     utility,
#     FieldSchema,
#     CollectionSchema,
#     DataType,
#     Collection,
# )
# 
# fields = [
#     # Use auto generated id as primary key
#     FieldSchema(
#         name="pk", dtype=DataType.VARCHAR, is_primary=True, auto_id=True, max_length=100
#     ),
#     # Store the original text to retrieve based on semantically distance
#     FieldSchema(name="model_id", dtype=DataType.INT32),
#     FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=5000),
#     # Milvus now supports both sparse and dense vectors,
#     # we can store each in a separate field to conduct hybrid search on both vectors
#     FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
#     FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=dense_dim),
# ]
# schema = CollectionSchema(fields, enable_dynamic_field=True)
# 
# col_name = "manual"
# if utility.has_collection(col_name, using=MILVUS_ALIAS):
#     Collection(col_name, using=MILVUS_ALIAS).drop()
# col = Collection(col_name, schema, consistency_level="Strong", using=MILVUS_ALIAS)
# 
# sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
# col.create_index("sparse_vector", sparse_index)
# dense_index = {"index_type": "AUTOINDEX", "metric_type": "IP"}
# col.create_index("dense_vector", dense_index)
# col.load()


# Save Data

In [198]:
# 1. PDF에서 텍스트 추출
docs = pdf_load('../../data/documents/genesis-gv60-manual-kor.pdf')
if not docs:
    raise ValueError("Failed to extract text from PDF.")

car_id = 14

In [199]:
docs['contents'][0]['text_combine']

'JW_GV60_KO.book  Page 1  \n안전 및 차량 손상 경고\n본 취급설명서는 고객 및 차량의 안전과 관련한 심각한 위험과 제품 사용에 대한 올바른 정보\n를 사전에 알리는 안전경고 표시입니다. 지시 사항은 반드시 숙지하여 지켜주십시오.\n경고, 주의 표시 경고,  주의가  있는  문장  및  진하게  표시되어  있는  부분은  특히  유\n선택 또는 미장착\n사양 표시\n념하십시오.\n (cid:2737)(cid:2628)\n 차량이 고장나거나 손상될 우려가 있는 경우의 주의 표시입니다.\n(cid:1155)(cid:1162)\n 사람이 다치거나 사망의 우려가 있는 경우의 경고 표시입니다.\n (cid:2444)(cid:2439)(cid:1601)(cid:1245)\n 차량 용어 또는 추가 설명이 필요한 정보 표시입니다.\n안전을 위해 반드시 지켜야 하는 금지 표시입니다.\n본 취급설명서는 모든 트림 모델 및 선택 사양을 포함하여 설명하\n고 있습니다.\n따라서 고객님 차량에 장착되지 않은 사양이 설명 될 수 있습니다. \n'

In [200]:
# 2. 텍스트 전처리
for doc in docs['contents']:
    doc['processed_text'] = clean_text(doc['text_combine'])

In [201]:
# 3. 텍스트 청킹
for doc in docs['contents']:
    doc['chunks'] = chunk_text(doc['processed_text'], chunk_size=100, overlap=20, max_length=2500)

In [202]:
# 4. Embedding
for index, doc in enumerate(docs['contents']):
    print(index)
    if doc['chunks']:
        doc['embedding'] = generate_embedding(doc['chunks'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [203]:
if not docs['contents'][3].get('embedding'):
    print('있다.')
else:
    print('없다')

없다


In [204]:
from RAG.database.milvus_connector import get_collection
collection = Collection('manual', using=MILVUS_ALIAS)

In [205]:
import numpy as np

for doc_index, doc in enumerate(docs['contents']):  # doc_index로 페이지 넘버 추적
    if not doc.get('embedding'):
        continue
        
    dense_embedding = doc['embedding']['dense']
    sparse_embedding = doc['embedding']['sparse']
    
    data=[]
    for i in range(0, len(dense_embedding), 1):
        entity = {
            "model_id": car_id,
            "text" : doc['chunks'][i],
            "sparse_vector": sparse_embedding[i:i+1],
            "dense_vector": np.array(dense_embedding[i]),
            "page": doc_index
        }
        data.append(entity)
        
        # 메타데이터를 포함하여 삽입
    collection.insert(data=data)
    
    print(f"{doc_index} : Number of entities inserted: {collection.num_entities}" )


0 : Number of entities inserted: 22995
1 : Number of entities inserted: 22995
2 : Number of entities inserted: 22995
3 : Number of entities inserted: 22995
4 : Number of entities inserted: 22995
5 : Number of entities inserted: 22995
6 : Number of entities inserted: 22995
7 : Number of entities inserted: 22995
8 : Number of entities inserted: 22995
9 : Number of entities inserted: 22995
10 : Number of entities inserted: 22995
11 : Number of entities inserted: 22995
12 : Number of entities inserted: 22995
13 : Number of entities inserted: 22995
14 : Number of entities inserted: 22995
15 : Number of entities inserted: 22995
16 : Number of entities inserted: 22995
17 : Number of entities inserted: 22995
18 : Number of entities inserted: 22995
19 : Number of entities inserted: 22995
20 : Number of entities inserted: 22995
21 : Number of entities inserted: 22995
22 : Number of entities inserted: 22995
23 : Number of entities inserted: 22995
24 : Number of entities inserted: 22995
25 : Numbe

In [206]:
for chunk in docs['contents'][487]['chunks']:
    print(len(chunk))

397
345
349
299
17


In [207]:
max_len = 0
for doc_index, doc in enumerate(docs['contents']):
    for chunk in doc['chunks']:
        if max_len < len(chunk):
            max_len = len(chunk)
            
max_len

2494